In [54]:
import numpy as np
import time
import pandas as pd
from pathlib import Path
import sys
import json
import datetime
from pre_process_neuralynx import *
from robust_stats import *


In [5]:
# make sure you are processing the right tasks by looking at the file.
#PreProcessingTable_Li_12_10_2018
AnimalID = 'Li'
#date_obj = '12_10_2018'
date_str = '12_10_2018'
#date_str= "%s_%s_%s" % (date_obj.month,date_obj.day,date_obj.year)

TaskDir = Path('../PreProcessing/TasksDir/')

with open(str(TaskDir) + '/PreProcessingTable_{}_{}.json'.format(AnimalID,date_str), 'r') as f:
        task_table = json.load(f)

In [33]:
x = TaskDir / ('PreProcessingTable_{}_{}.json'.format(AnimalID,date_str))
x.parent / (x.stem + '_Completed.json')

PosixPath('../PreProcessing/TasksDir/PreProcessingTable_Li_12_10_2018_Completed.json')

In [17]:
task_table.keys()

dict_keys(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44'])

In [19]:
taskID=3
session=task_table[str(taskID)]
session.keys()
session['sp']

'/scratch/users/alexg8/Data/Li/Li_OF_061118_Results'

In [13]:
task_list=session['Files']
task_list

{'1': {'filenames': ['/scratch/users/alexg8/Data/Li/Li_OF_061118/CSC1a.ncs',
   '/scratch/users/alexg8/Data/Li/Li_OF_061118/CSC1b.ncs',
   '/scratch/users/alexg8/Data/Li/Li_OF_061118/CSC1c.ncs',
   '/scratch/users/alexg8/Data/Li/Li_OF_061118/CSC1d.ncs'],
  'sp': '/scratch/users/alexg8/Data/Li/Li_OF_061118_Results',
  'subSessionID': '0000',
  'tt_id': '1',
  'type': 'tt'},
 '10': {'filenames': ['/scratch/users/alexg8/Data/Li/Li_OF_061118/CSC10a.ncs',
   '/scratch/users/alexg8/Data/Li/Li_OF_061118/CSC10b.ncs',
   '/scratch/users/alexg8/Data/Li/Li_OF_061118/CSC10c.ncs',
   '/scratch/users/alexg8/Data/Li/Li_OF_061118/CSC10d.ncs'],
  'sp': '/scratch/users/alexg8/Data/Li/Li_OF_061118_Results',
  'subSessionID': '0000',
  'tt_id': '10',
  'type': 'tt'},
 '11': {'filenames': ['/scratch/users/alexg8/Data/Li/Li_OF_061118/CSC11a.ncs',
   '/scratch/users/alexg8/Data/Li/Li_OF_061118/CSC11b.ncs',
   '/scratch/users/alexg8/Data/Li/Li_OF_061118/CSC11c.ncs',
   '/scratch/users/alexg8/Data/Li/Li_OF_061

In [37]:
task = task_list[str(1)]
b = np.fromfile('filt_b.dat',dtype='float',sep=',')
a = np.fromfile('filt_a.dat',dtype='float',sep=',')

In [38]:
# Unpack the tetrode information.
tt_id = task['tt_id']
csc_files = task['filenames']
sp = Path(task['sp'])
ss = task['subSessionID']

if ss=='0000':
    ttFile = 'tt_{}'.format(tt_id)
else:
    ttFile = 'tt_{}_{}'.format(tt_id,ss)

In [40]:
# load first channel
f =[]
f.append(csc_files[0])
try:
    h1  = get_header(f[0])
    sig,time_stamps = get_csc(f[0])
    nSamps = len(sig)
except:
    sys.exit('Could not read first channel. Aborting.')

In [45]:
# create information file
AmpPercentileThr=0.975
info  = {'tetrodeID':[tt_id],'RefChan':h1['RefChan'],
'fs':h1['fs'],'AmpPercentileThr':AmpPercentileThr,'nSamps':nSamps,
'date_processed': date_str, 'subSessionID':ss}

In [47]:
chan_id=0

In [49]:
chan_id_str = str(chan_id)

# get channel specific info
h2  = get_header(f[chan_id])
info['AD'+'_'+chan_id_str]=float(h2['AD'])
info['InputRange'+'_'+chan_id_str]=h2['InputRange']
info['AmpRejThr'+'_'+chan_id_str] = h2['InputRange']*AmpPercentileThr
info['date_created'+'_'+chan_id_str]=get_file_date(f[chan_id])
rejThr= h2['InputRange']*AmpPercentileThr


In [59]:

## Step 1. Filter.
t1=time.time()
fsig = FilterCSC(sig,b,a)
t2=time.time()
print(t2-t1)

1545105304.146149


In [62]:
def get_sig_stats(sig):
    out = {}
    out['std'] = np.std(sig)
    out['rms'] = np.sqrt(np.nanmean(np.square(sig)))
    out['mad'] = mad(sig)
    out['min'] = np.nanmin(sig)
    out['max'] = np.nanmax(sig)
    return out

In [65]:
sig_stats = get_sig_stats(sig)
fsig_stats = get_sig_stats(fsig)

{'AD_0': 4.57763671875e-08,
 'AmpPercentileThr': 0.975,
 'AmpRejThr_0': 1462.5,
 'InputRange_0': 1500,
 'RefChan': 19,
 'date_created_0': '10_31_2018',
 'date_processed': '12_10_2018',
 'fs': 32000,
 'nSamps': 23124992,
 'subSessionID': '0000',
 'tetrodeID': ['1']}

In [64]:
print(sig_stats)
print(fsig_stats)

{'std': 43.169163, 'rms': 43.169216, 'mad': 20.004272, 'min': -887.46643, 'max': 860.73303}
{'std': 6.122144309268347, 'rms': 6.122144309268438, 'mad': 4.1288697337287275, 'min': -58.316580413437734, 'max': 36.27634320661544}


In [71]:
info['sig_stats_'+chan_id_str] = sig_stats
info['sig_stats_'+chan_id_str]

{'mad': 20.004272,
 'max': 860.73303,
 'min': -887.46643,
 'rms': 43.169216,
 'std': 43.169163}

In [61]:
sig_mad = mad(sig)
fsig_mad = mad(fsig)
sig_mad, fsig_mad

(20.004272, 4.1288697337287275)

In [79]:


"{0:0.3e}".format(10000/nSamps)

'4.324e-04'